In [408]:
%%shell
jupyter nbconvert --to html /content/Hackathon_Restaurant_Turnover_Prediction.ipynb

[NbConvertApp] Converting notebook /content/Hackathon_Restaurant_Turnover_Prediction.ipynb.ipynb to html
[NbConvertApp] Writing 752246 bytes to /content/Hackathon_Restaurant_Turnover_Prediction.ipynb.html


In [409]:
# Make the Python code more structured automatically
#%load_ext nb_black

# Libraries to help with reading and manipulating data
import numpy as np
import pandas as pd

# Libraries to help with data visualization
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("darkgrid")

# This is so I can see the entire dataframe when I print it
pd.set_option("display.max_columns", None)

# to split the data into train and test
from sklearn.model_selection import train_test_split

# to build linear regression_model
from sklearn.linear_model import LinearRegression

# to check model performance
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# to build linear regression_model using statsmodels
import statsmodels.api as sm

# to check for homoscedasticity
import statsmodels.stats.api as sms
from statsmodels.compat import lzip

# to check for multicollinearity
from statsmodels.stats.outliers_influence import variance_inflation_factor

# to check for normality
import scipy.stats as stats

# To build model for prediction
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

# To tune different models
from sklearn.model_selection import GridSearchCV

import pandas as pd
import numpy as np
from sklearn.linear_model import RidgeCV

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [410]:
train = pd.read_csv("Train_dataset.csv")
train.head()

,Registration Number,Annual Turnover,Cuisine,City,Restaurant Location,Opening Day of Restaurant,Facebook Popularity Quotient,Endorsed By,Instagram Popularity Quotient,Fire Audit,Liquor License Obtained,Situated in a Multi Complex,Dedicated Parking,Open Sitting Available,Resturant Tier,Restaurant Type,Restaurant Theme,Restaurant Zomato Rating,Restaurant City Tier,Order Wait Time,Staff Responsivness,Value for Money,Hygiene Rating,Food Rating,Overall Restaurant Rating,Live Music Rating,Comedy Gigs Rating,Value Deals Rating,Live Sports Rating,Ambience,Lively,Service,Comfortablility,Privacy
0,60001,42000000,"indian,irish",Bangalore,Near Business Hub,14-02-2009,84.3,Not Specific,95.8,1,1,1,0,1,2.0,Bar,Arabian,3,0,2,5,5,5,7,10.0,4.0,NaN,NaN,NaN,8.0,8,6,6,6
1,60002,50000000,"indian,irish",Indore,Near Party Hub,29-09-2008,85.4,Tier A Celebrity,85.0,1,1,1,1,0,2.0,Bar,Greek,3,0,6,6,6,7,8,9.0,NaN,4.0,NaN,NaN,5.0,7,7,3,8
2,60003,32500000,"tibetan,italian",Chennai,Near Business Hub,30-07-2011,85.0,Tier A Celebrity,68.2,1,1,1,1,0,2.0,Bar,90's,3,0,2,6,5,3,7,8.0,3.0,NaN,NaN,NaN,7.0,10,5,2,8
3,60004,110000000,"turkish,nigerian",Gurgaon,Near Party Hub,30-11-2008,85.6,Tier A Celebrity,83.6,0,1,1,1,1,1.0,Bar,Arabian,3,1,7,6,5,6,8,9.0,6.0,NaN,NaN,NaN,7.0,7,4,3,5
4,60005,20000000,"irish,belgian",Manesar,Near Party Hub,22-02-2010,NaN,Tier A Celebrity,76.8,1,1,1,1,1,NaN,Bar,Greek,3,0,10,5,6,4,8,6.0,NaN,2.0,NaN,NaN,NaN,6,2,4,6


In [411]:
test = pd.read_csv("Test_dataset.csv")
test.head()

,Registration Number,Cuisine,City,Restaurant Location,Opening Day of Restaurant,Facebook Popularity Quotient,Endoresed By,Instagram Popularity Quotient,Fire Audit,Liquor License Obtained,Situated in a Multi Complex,Dedicated Parking,Open Sitting Available,Resturant Tier,Restaurant Type,Restaurant Theme,Restaurant Zomato Rating,Restaurant City Tier,Order Wait Time,Staff Responsivness,Value for Money,Hygiene Rating,Food Rating,Overall Restaurant Rating,Live Music Rating,Comedy Gigs Rating,Value Deals Rating,Live Sports Rating,Ambience,Lively,Service,Comfortablility,Privacy
0,20001,"tibetan,italian",Bangalore,Near Business Hub,13-07-2010,78.00,Tier A Celebrity,69.00,1,1,1,1,0,2,Bar,Greek,3,0,10,5,3,5,8,6.0,NaN,2.0,NaN,NaN,5,2,2,6,0
1,20002,"tibetan,italian",Hyderabad,Near Party Hub,05-09-2011,89.17,Not Specific,96.00,1,1,1,1,1,2,Bar,Nature,4,0,10,4,5,6,8,9.0,3.0,NaN,NaN,6.0,7,8,5,1,8
2,20003,"algerian,belgian",Hyderabad,Near Party Hub,12-04-2011,84.00,Not Specific,86.00,1,1,1,0,1,2,Bar,Greek,2,0,6,5,5,6,9,6.0,3.0,NaN,NaN,NaN,9,7,5,1,5
3,20004,"tibetan,greek",-1,Near Party Hub,16-01-2005,79.38,Not Specific,74.40,0,1,1,1,1,2,Gastro Bar,90's,3,0,10,5,5,5,9,9.0,6.0,NaN,NaN,NaN,6,7,6,4,7
4,20005,"cuban,british",Pune,Near Party Hub,10-11-2008,84.67,Not Specific,86.46,1,1,1,0,0,1,Bar,Mise En,4,1,7,5,5,7,8,NaN,NaN,NaN,NaN,NaN,4,4,7,7,3


In [412]:
#Rename Columns
train = train.rename(columns={"Resturant Tier": "Restaurant Tier"})
train = train.rename(columns={"Staff Responsivness": "Staff Responsiveness"})
train = train.rename(columns={"Comfortablility": "Comfortability"})

In [413]:
#Rename Columns
test = test.rename(columns={"Endoresed By": "Endorsed By"})
test = test.rename(columns={"Resturant Tier": "Restaurant Tier"})
test = test.rename(columns={"Staff Responsivness": "Staff Responsiveness"})
test = test.rename(columns={"Comfortablility": "Comfortability"})

In [414]:
train.shape,test.shape

((3493, 34), (500, 33))

In [415]:
df=pd.concat([train,test])
df.head()

,Registration Number,Annual Turnover,Cuisine,City,Restaurant Location,Opening Day of Restaurant,Facebook Popularity Quotient,Endorsed By,Instagram Popularity Quotient,Fire Audit,Liquor License Obtained,Situated in a Multi Complex,Dedicated Parking,Open Sitting Available,Restaurant Tier,Restaurant Type,Restaurant Theme,Restaurant Zomato Rating,Restaurant City Tier,Order Wait Time,Staff Responsiveness,Value for Money,Hygiene Rating,Food Rating,Overall Restaurant Rating,Live Music Rating,Comedy Gigs Rating,Value Deals Rating,Live Sports Rating,Ambience,Lively,Service,Comfortability,Privacy
0,60001,42000000.0,"indian,irish",Bangalore,Near Business Hub,14-02-2009,84.3,Not Specific,95.8,1,1,1,0,1,2.0,Bar,Arabian,3,0,2,5,5,5,7,10.0,4.0,NaN,NaN,NaN,8.0,8,6,6,6
1,60002,50000000.0,"indian,irish",Indore,Near Party Hub,29-09-2008,85.4,Tier A Celebrity,85.0,1,1,1,1,0,2.0,Bar,Greek,3,0,6,6,6,7,8,9.0,NaN,4.0,NaN,NaN,5.0,7,7,3,8
2,60003,32500000.0,"tibetan,italian",Chennai,Near Business Hub,30-07-2011,85.0,Tier A Celebrity,68.2,1,1,1,1,0,2.0,Bar,90's,3,0,2,6,5,3,7,8.0,3.0,NaN,NaN,NaN,7.0,10,5,2,8
3,60004,110000000.0,"turkish,nigerian",Gurgaon,Near Party Hub,30-11-2008,85.6,Tier A Celebrity,83.6,0,1,1,1,1,1.0,Bar,Arabian,3,1,7,6,5,6,8,9.0,6.0,NaN,NaN,NaN,7.0,7,4,3,5
4,60005,20000000.0,"irish,belgian",Manesar,Near Party Hub,22-02-2010,NaN,Tier A Celebrity,76.8,1,1,1,1,1,NaN,Bar,Greek,3,0,10,5,6,4,8,6.0,NaN,2.0,NaN,NaN,NaN,6,2,4,6


In [416]:
# Returns Statistical summary of the data for Numerical Datatype
df.describe(exclude=[object]).T

,count,mean,std,min,25%,50%,75%,max
Registration Number,3993.0,5.655084e+04,1.376814e+04,20001.0,60499.00,61497.00,6.249500e+04,6.349300e+04
Annual Turnover,3493.0,3.072571e+07,2.165125e+07,3500000.0,18000000.00,30000000.00,3.700000e+07,4.000000e+08
Facebook Popularity Quotient,3894.0,7.788768e+01,9.861789e+00,43.0,71.66,79.00,8.566750e+01,9.776000e+01
Instagram Popularity Quotient,3937.0,7.444895e+01,1.099066e+01,40.0,66.00,74.33,8.260000e+01,9.870000e+01
Fire Audit,3993.0,7.918858e-01,4.060098e-01,0.0,1.00,1.00,1.000000e+00,1.000000e+00
Liquor License Obtained,3993.0,9.879790e-01,1.089931e-01,0.0,1.00,1.00,1.000000e+00,1.000000e+00
Situated in a Multi Complex,3993.0,8.056599e-01,3.957414e-01,0.0,1.00,1.00,1.000000e+00,1.000000e+00
Dedicated Parking,3993.0,8.026546e-01,3.980450e-01,0.0,1.00,1.00,1.000000e+00,1.000000e+00
Open Sitting Available,3993.0,8.006511e-01,3.995609e-01,0.0,1.00,1.00,1.000000e+00,1.000000e+00
Restaurant Tier,3944.0,1.927231e+00,2.597895e-01,1.0,2.00,2.00,2.000000e+00,2.000000e+00


In [417]:
# Returns Statistical summary of the data for object Datatype
df.describe(exclude=[np.number]).T

,count,unique,top,freq
Cuisine,3993,20,"tibetan,greek",801
City,3993,334,Bangalore,627
Restaurant Location,3993,2,Near Party Hub,3036
Opening Day of Restaurant,3993,1870,27-12-2009,11
Endorsed By,3993,3,Not Specific,2260
Restaurant Type,3993,4,Bar,3695
Restaurant Theme,3993,34,Arabian,1592


In [418]:
df.duplicated().sum()

0

In [419]:
df.isnull().sum()

Registration Number                 0
Annual Turnover                   500
Cuisine                             0
City                                0
Restaurant Location                 0
Opening Day of Restaurant           0
Facebook Popularity Quotient       99
Endorsed By                         0
Instagram Popularity Quotient      56
Fire Audit                          0
Liquor License Obtained             0
Situated in a Multi Complex         0
Dedicated Parking                   0
Open Sitting Available              0
Restaurant Tier                    49
Restaurant Type                     0
Restaurant Theme                    0
Restaurant Zomato Rating            0
Restaurant City Tier                0
Order Wait Time                     0
Staff Responsiveness                0
Value for Money                     0
Hygiene Rating                      0
Food Rating                         0
Overall Restaurant Rating         246
Live Music Rating                 867
Comedy Gigs 

Data -Preprocessing**

In [420]:
#Missing value analysis
null_train=pd.DataFrame()
null_train['Features']=df.isnull().sum().index
null_train['Null values']=df.isnull().sum().values
null_train['% Null values']=(df.isnull().sum().values / df.shape[0])*100
null_train.sort_values(by='% Null values',ascending=False)

,Features,Null values,% Null values
28,Live Sports Rating,3759,94.139745
27,Value Deals Rating,3092,77.435512
26,Comedy Gigs Rating,2853,71.450038
25,Live Music Rating,867,21.712998
1,Annual Turnover,500,12.521913
24,Overall Restaurant Rating,246,6.160781
6,Facebook Popularity Quotient,99,2.479339
8,Instagram Popularity Quotient,56,1.402454
14,Restaurant Tier,49,1.227148
29,Ambience,25,0.626096


In [421]:
# We will drop the columns which are having more than 50% NULL Values
null_columns=null_train[null_train['% Null values']>=50]['Features']
df.drop(columns=null_columns,inplace=True)
np.array(null_columns)     ##Columns that are removed

array(['Comedy Gigs Rating', 'Value Deals Rating', 'Live Sports Rating'],
      dtype=object)

In [422]:
#Missing value Treatment
numeric_cols_train = df.select_dtypes(include=np.number).columns.tolist()
df[numeric_cols_train] = df[numeric_cols_train].apply(lambda x: x.fillna(x.median()), axis=0)

category_cols_train = df.select_dtypes(include='object').columns.tolist()
df[category_cols_train] = df[category_cols_train].apply(lambda x: x.fillna(x.value_counts().index[0]), axis=0)


In [423]:
df['Opening Day of Restaurant']=df['Opening Day of Restaurant'].str.slice(-4)
yr=df.groupby(['Opening Day of Restaurant'])[['Annual Turnover']].mean().sort_values('Annual Turnover')
yr


,Annual Turnover
Opening Day of Restaurant,
2000,1.100000e+07
1996,1.200000e+07
2014,2.700000e+07
2013,2.705172e+07
2011,2.782924e+07
2012,2.789358e+07
2010,2.889980e+07
2001,3.000000e+07
2002,3.060000e+07


In [424]:
d={}
c=1
for i in yr.index:
    d[i]=c
    c=c+1
print(d)

{'2000': 1, '1996': 2, '2014': 3, '2013': 4, '2011': 5, '2012': 6, '2010': 7, '2001': 8, '2002': 9, '2009': 10, '2008': 11, '2005': 12, '2004': 13, '2007': 14, '2006': 15, '2016': 16, '2003': 17}


In [425]:
df['Opening Day of Restaurant']=df['Opening Day of Restaurant'].map(d)

In [426]:
tmp=df.groupby(['Cuisine'])[['Annual Turnover']].mean().sort_values('Annual Turnover')
tmp


,Annual Turnover
Cuisine,
"irish,belgian",2.360000e+07
"chinese,salvadorian",2.573118e+07
"welsh,thai",2.658189e+07
"cuban,british",2.671875e+07
"turkish,sapnish",2.692188e+07
"swedish,greek",2.742157e+07
"nigerian,cajun",2.742857e+07
"peruvian,cuban",2.815299e+07
"algerian,korean",2.925000e+07


In [427]:
d={}
c=1
for i in tmp.index:
    d[i]=c
    c=c+1
print(d)


{'irish,belgian': 1, 'chinese,salvadorian': 2, 'welsh,thai': 3, 'cuban,british': 4, 'turkish,sapnish': 5, 'swedish,greek': 6, 'nigerian,cajun': 7, 'peruvian,cuban': 8, 'algerian,korean': 9, 'algerian,belgian': 10, 'algerian,algerian': 11, 'british,belgian': 12, 'british,japanese': 13, 'tibetan,italian': 14, 'indian,irish': 15, 'polish,jewish': 16, 'tibetan,greek': 17, 'japanese,thai': 18, 'turkish,nigerian': 19, 'hawaiian,latvian': 20}


In [428]:
df['Cuisine']=df['Cuisine'].map(d)

In [429]:
temp=df.groupby(['Restaurant Theme'])[['Annual Turnover']].mean().sort_values('Annual Turnover')
temp

,Annual Turnover
Restaurant Theme,
Spoon,1.000000e+07
Grand,1.000000e+07
Umami,1.483333e+07
Cinnamon,2.000000e+07
Resca,2.100000e+07
Ambiance,2.225000e+07
TheGem,2.458333e+07
Picante,2.550000e+07
Marco,2.560000e+07


In [430]:
d={}
c=1
for i in temp.index:
    d[i]=c
    c=c+1
print(d)

{'Spoon': 1, 'Grand': 2, 'Umami': 3, 'Cinnamon': 4, 'Resca': 5, 'Ambiance': 6, 'TheGem': 7, 'Picante': 8, 'Marco': 9, 'Belise': 10, 'Moonrise': 11, 'Parsi': 12, 'Delicio': 13, 'Chinese': 14, 'Greek': 15, 'Benedicta': 16, 'Italica': 17, "90's": 18, 'Mise En': 19, 'Arabian': 20, 'Nature': 21, 'Flavors': 22, 'Petit': 23, 'Bakery': 24, 'Bettaso': 25, 'Carbone': 26, 'Molino': 27, 'Sage': 28, 'Cafepro': 29, 'Downtown': 30, 'Savory': 31, 'English': 32, 'Piquant': 33, 'LaCuisine': 34}


In [431]:
df['Restaurant Theme']=df['Restaurant Theme'].map(d)

In [432]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3993 entries, 0 to 499
Data columns (total 31 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Registration Number            3993 non-null   int64  
 1   Annual Turnover                3993 non-null   float64
 2   Cuisine                        3993 non-null   int64  
 3   City                           3993 non-null   object 
 4   Restaurant Location            3993 non-null   object 
 5   Opening Day of Restaurant      3993 non-null   int64  
 6   Facebook Popularity Quotient   3993 non-null   float64
 7   Endorsed By                    3993 non-null   object 
 8   Instagram Popularity Quotient  3993 non-null   float64
 9   Fire Audit                     3993 non-null   int64  
 10  Liquor License Obtained        3993 non-null   int64  
 11  Situated in a Multi Complex    3993 non-null   int64  
 12  Dedicated Parking              3993 non-null   in

In [433]:
# Normalize the variables and compute the average normalized value for each row
df['Order Wait Time'] = (df['Order Wait Time'] - 1) / 9
df['Staff Responsiveness'] = (df['Staff Responsiveness'] - 1) / 7
df['Value for Money'] = (df['Value for Money'] - 1) / 6
# Normalize the variables and compute the average normalized value for each row
df['Hygiene Rating'] = (df['Hygiene Rating'] - 1) / 9
df['Food Rating'] = (df['Food Rating'] - 1) / 9
# Normalize the variables and compute the average normalized value for each row
df['Ambience'] = (df['Ambience'] - 0) / 9
df['Lively'] = (df['Lively'] - 1) / 9
df['Service'] = (df['Service'] - 0) / 19
df['Comfortability'] = (df['Comfortability'] - 0) / 9
df['Privacy'] = (df['Privacy'] - 0) / 9
df['Overall Restaurant Rating']= (df['Overall Restaurant Rating'] - 1) / 9
df['Lively and Privacy']=(df['Lively']+df['Privacy'])/2
df = df.drop(['Lively', 'Privacy'], axis=1)


In [434]:
# Normalize Facebook Popularity Quotient and Instagram Popularity Quotient
df['Facebook Popularity Quotient'] = (df['Facebook Popularity Quotient'] - df['Facebook Popularity Quotient'].min()) / (df['Facebook Popularity Quotient'].max() - df['Facebook Popularity Quotient'].min())
df['Instagram Popularity Quotient'] = (df['Instagram Popularity Quotient'] - df['Instagram Popularity Quotient'].min()) / (df['Instagram Popularity Quotient'].max() - df['Instagram Popularity Quotient'].min())

# Combine normalized Facebook Popularity Quotient and Instagram Popularity Quotient into one variable
df['Media Popularity'] = (df['Facebook Popularity Quotient'] + df['Instagram Popularity Quotient']) / 2

# Drop the original Facebook Popularity Quotient and Instagram Popularity Quotient columns
df = df.drop(['Facebook Popularity Quotient', 'Instagram Popularity Quotient'], axis=1)


In [435]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3993 entries, 0 to 499
Data columns (total 29 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Registration Number          3993 non-null   int64  
 1   Annual Turnover              3993 non-null   float64
 2   Cuisine                      3993 non-null   int64  
 3   City                         3993 non-null   object 
 4   Restaurant Location          3993 non-null   object 
 5   Opening Day of Restaurant    3993 non-null   int64  
 6   Endorsed By                  3993 non-null   object 
 7   Fire Audit                   3993 non-null   int64  
 8   Liquor License Obtained      3993 non-null   int64  
 9   Situated in a Multi Complex  3993 non-null   int64  
 10  Dedicated Parking            3993 non-null   int64  
 11  Open Sitting Available       3993 non-null   int64  
 12  Restaurant Tier              3993 non-null   float64
 13  Restaurant Type    

In [436]:
Categorical_columns=["Restaurant Location","Endorsed By","Restaurant Type"]
df = pd.get_dummies(df, columns=Categorical_columns,drop_first=True)

In [437]:
# Outlier Treament Function


def treat_outliers(df, col):
    """
    Treats outliers in a variable

    df: dataframe
    col: dataframe column
    """
    Q1 = df[col].quantile(0.25)  # 25th quantile
    Q3 = df[col].quantile(0.75)  # 75th quantile
    IQR = Q3 - Q1
    Lower_Whisker = Q1 - 1.5 * IQR
    Upper_Whisker = Q3 + 1.5 * IQR

    # all the values smaller than Lower_Whisker will be assigned the value of Lower_Whisker
    # all the values greater than Upper_Whisker will be assigned the value of Upper_Whisker
    df[col] = np.clip(df[col], Lower_Whisker, Upper_Whisker)

    return df


def treat_outliers_all(df, col_list):
    """
    Treat outliers in a list of variables

    df: dataframe
    col_list: list of dataframe columns
    """
    for c in col_list:
        df = treat_outliers(df, c)

    return df

In [438]:
binomial_cols=["Fire Audit","Liquor License Obtained","Situated in a Multi Complex","Dedicated Parking",
"Open Sitting Available","Restaurant Tier",'Restaurant City Tier']

In [439]:
#Outlier Treatment
numerical_cols = [col for col in train.select_dtypes(include=np.number).columns.tolist() if col not in binomial_cols]
print(numerical_cols)
data = treat_outliers_all(train, numerical_cols)

['Registration Number', 'Annual Turnover', 'Facebook Popularity Quotient', 'Instagram Popularity Quotient', 'Restaurant Zomato Rating', 'Order Wait Time', 'Staff Responsiveness', 'Value for Money', 'Hygiene Rating', 'Food Rating', 'Overall Restaurant Rating', 'Live Music Rating', 'Comedy Gigs Rating', 'Value Deals Rating', 'Live Sports Rating', 'Ambience', 'Lively', 'Service', 'Comfortability', 'Privacy']


In [440]:
#drop city column
df=df.drop('City', axis=1)

In [441]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3993 entries, 0 to 499
Data columns (total 31 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Registration Number                       3993 non-null   int64  
 1   Annual Turnover                           3993 non-null   float64
 2   Cuisine                                   3993 non-null   int64  
 3   Opening Day of Restaurant                 3993 non-null   int64  
 4   Fire Audit                                3993 non-null   int64  
 5   Liquor License Obtained                   3993 non-null   int64  
 6   Situated in a Multi Complex               3993 non-null   int64  
 7   Dedicated Parking                         3993 non-null   int64  
 8   Open Sitting Available                    3993 non-null   int64  
 9   Restaurant Tier                           3993 non-null   float64
 10  Restaurant Theme                     

**Data Modelling**

In [442]:
X = df.drop(["Annual Turnover","Endorsed By_Not Specific"], axis=1)
y = df["Annual Turnover"]

In [443]:
X.shape,y.shape

((3993, 29), (3993,))

In [444]:
x_train=sm.add_constant(X.iloc[0:3493,:],has_constant='add')

In [445]:
x_test=sm.add_constant(X.iloc[3493:,:],has_constant='add')

In [446]:
y_train=y.iloc[0:3493]

In [447]:
x_train.shape,x_test.shape,y_train.shape

((3493, 30), (500, 30), (3493,))

In [448]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
# to compute VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Calculate the VIF values for each predictor
def calc_vif(X):
    vif = pd.DataFrame()
    vif['variable'] = X.columns
    vif['VIF'] = [ variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif

vif = calc_vif(x_train)
print(vif)



                                    variable          VIF
0                                      const  4161.342303
1                        Registration Number     1.005279
2                                    Cuisine     1.033433
3                  Opening Day of Restaurant     1.188267
4                                 Fire Audit     1.008998
5                    Liquor License Obtained     1.231554
6                Situated in a Multi Complex     1.006523
7                          Dedicated Parking     1.010162
8                     Open Sitting Available     1.006170
9                            Restaurant Tier     1.137831
10                          Restaurant Theme     1.085243
11                  Restaurant Zomato Rating     1.265761
12                      Restaurant City Tier     1.058922
13                           Order Wait Time     1.007070
14                      Staff Responsiveness     1.421659
15                           Value for Money     1.530905
16            

In [449]:
# Identify the predictors with high VIF values
high_vif_vars = vif[vif['VIF'] > 5]['variable'].values
print(high_vif_vars)



['const']


In [450]:
# function to compute adjusted R-squared
def adj_r2_score(predictors, targets, predictions):
    r2 = r2_score(targets, predictions)
    n = predictors.shape[0]
    k = predictors.shape[1]
    return 1 - ((1 - r2) * (n - 1) / (n - k - 1))


# function to compute MAPE
def mape_score(targets, predictions):
    return np.mean(np.abs(targets - predictions) / targets) * 100


# function to compute different metrics to check performance of a regression model
def model_performance_regression(model, predictors, target):
    """
    Function to compute different metrics to check regression model performance

    model: regressor
    predictors: independent variables
    target: dependent variable
    """

    # predicting using the independent variables
    pred = model.predict(predictors)

    r2 = r2_score(target, pred)  # to compute R-squared
    adjr2 = adj_r2_score(predictors, target, pred)  # to compute adjusted R-squared
    rmse = np.sqrt(mean_squared_error(target, pred))  # to compute RMSE
    mae = mean_absolute_error(target, pred)  # to compute MAE
    mape = mape_score(target, pred)  # to compute MAPE

    # creating a dataframe of metrics
    df_perf = pd.DataFrame(
        {
            "RMSE": rmse,
            "MAE": mae,
            "R-squared": r2,
            "Adj. R-squared": adjr2,
            "MAPE": mape,
        },
        index=[0],
    )

    return df_perf

In [451]:
x_train.shape,x_test.shape,y_train.shape

((3493, 30), (500, 30), (3493,))

In [452]:
olsmodel = sm.OLS(y_train,x_train).fit()
print(olsmodel.summary())

                            OLS Regression Results                            
Dep. Variable:        Annual Turnover   R-squared:                       0.158
Model:                            OLS   Adj. R-squared:                  0.151
Method:                 Least Squares   F-statistic:                     22.38
Date:                Sun, 02 Apr 2023   Prob (F-statistic):          2.43e-107
Time:                        05:28:26   Log-Likelihood:                -63655.
No. Observations:                3493   AIC:                         1.274e+05
Df Residuals:                    3463   BIC:                         1.276e+05
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------

In [453]:
# initial list of columns
predictors = x_train.copy()
cols = predictors.columns.tolist()
cols.remove('const')
# setting an initial max p-value
max_p_value = 1

while len(cols) > 0:
    # defining the train set
    x_train_aux = predictors[cols]

    # fitting the model
    model = sm.OLS(y_train, x_train_aux).fit()

    # getting the p-values and the maximum p-value
    p_values = model.pvalues
    max_p_value = max(p_values)

    # name of the variable with maximum p-value
    feature_with_p_max = p_values.idxmax()

    if max_p_value > 0.05:
        #print("The columns with P Value > 0.05", feature_with_p_max)
        cols.remove(feature_with_p_max)
    else:
        break

selected_features = cols
print(selected_features)

['Cuisine', 'Opening Day of Restaurant', 'Situated in a Multi Complex', 'Restaurant Tier', 'Restaurant Theme', 'Restaurant Zomato Rating', 'Restaurant City Tier', 'Staff Responsiveness', 'Hygiene Rating', 'Live Music Rating', 'Ambience', 'Comfortability', 'Media Popularity', 'Restaurant Location_Near Party Hub', 'Restaurant Type_Caffee']


In [454]:
x_train = x_train[selected_features]
x_test = x_test[selected_features]

**OLS Model**

In [455]:
olsmodel_final = sm.OLS(y_train,x_train).fit()
print(olsmodel_final.summary())

                                 OLS Regression Results                                
Dep. Variable:        Annual Turnover   R-squared (uncentered):                   0.720
Model:                            OLS   Adj. R-squared (uncentered):              0.718
Method:                 Least Squares   F-statistic:                              595.2
Date:                Sun, 02 Apr 2023   Prob (F-statistic):                        0.00
Time:                        05:28:27   Log-Likelihood:                         -63661.
No. Observations:                3493   AIC:                                  1.274e+05
Df Residuals:                    3478   BIC:                                  1.274e+05
Df Model:                          15                                                  
Covariance Type:            nonrobust                                                  
                                         coef    std err          t      P>|t|      [0.025      0.975]
-----------------

In [456]:
# Make predictions on the test set using the trained model
y_test_pred = olsmodel_final.predict(x_test)

# Create a DataFrame with the Registration Number and predicted Annual Turnover values
results = pd.DataFrame({"Registration Number": test["Registration Number"], "Annual Turnover": y_test_pred})

# Convert the Annual Turnover column to integers
results["Annual Turnover"] = results["Annual Turnover"].astype(int)

# Set the Registration Number column as the index
results.set_index("Registration Number", inplace=True)

# Export the results as a CSV file
results.to_csv("Best_Model_OLS_results.csv")


In [457]:
results

,Annual Turnover
Registration Number,
20001,26172557
20002,35431827
20003,25153117
20004,42014541
20005,46985590
...,...
20496,34492128
20497,31636063
20498,35122050
